In [1]:
! pip install --quiet transformers


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from sklearn.metrics import precision_score, recall_score
import pandas as pd
import numpy as np


In [4]:
test_data_processed = pd.read_csv("/content/gdrive/My Drive/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv")

In [5]:
display(test_data_processed)

,text,label,sentence_variation,%stop_word_total,mean_word_length,%adjectives_total
0,marshall plan test progress test progress whet...,0,11.491986,0.384845,5.544367,0.120638
1,promoting global regional security postcold wa...,0,14.830529,0.418864,5.314402,0.102434
2,womanhood peacemaking taking advantage unity c...,0,9.248099,0.374872,5.745659,0.102145
3,human rights universal western construct cruci...,0,17.593591,0.494980,4.988956,0.092369
4,challenges american foreign service rebuilding...,0,10.024878,0.371251,5.815926,0.099276
5,history dangerous history essential part cultu...,0,14.090764,0.456088,5.185629,0.127745
6,camp david accords bible promises blessed peac...,0,11.505633,0.355852,5.546147,0.098002
7,sudan conflict human rights year civil war eru...,0,9.078032,0.425243,4.935922,0.082524
8,1993 antanas mockus dropped pants mooned crowd...,0,40.511411,0.390716,5.487427,0.108317
9,standing human rights challenges facing foreig...,0,13.836479,0.369792,5.377083,0.103125


In [6]:
# Load the pre-trained XLNet model and tokenizer
model_name = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
texts = test_data_processed['text'].astype(str).tolist()  # Ensure texts are strings
labels = test_data_processed['label'].tolist()

In [8]:
# Tokenize the texts
tokenized_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']

In [9]:
# Convert the labels to tensors
label_tensors = torch.tensor(labels)

# Put the model in evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    # Forward pass
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_tensors)

    # Get the loss, logits, and predicted labels
    loss = outputs.loss
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).tolist()

# Calculate precision and recall
precision = precision_score(labels, predicted_labels)
recall = recall_score(labels, predicted_labels)

In [10]:
print("Loss: {:.4f}".format(loss.item()))
print("Precision: {:.2%}".format(precision))
print("Recall: {:.2%}".format(recall))

Loss: 0.7439
Precision: 54.55%
Recall: 90.00%
